# Emo_Bert_Training

This Notebook has been run on Google Colab with a Premium Account

**Input :** Solution.pkl

**main  :** Training and testing our pre-trained Bert Classifier for emotions on our novel dataset.
        Running Vader


In [1]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 4.8MB/s eta 0:00:01
     |████████████████████████████████| 890kB 14.6MB/s 
     |████████████████████████████████| 1.1MB 24.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5d9df22d9689653892fa882be37713613372848dcf63563119206ba346ebe8f7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf

tf.compat.v1.enable_eager_execution()

In [ ]:
# After eager execution is enabled, operations are executed as they are
# defined and Tensor objects hold concrete values, which can be accessed as
# numpy.ndarray`s through the numpy() method.
assert tf.multiply(6, 7).numpy() == 42

In [4]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
labels = ['afraid',
            'angry',
            'annoyed',
            'anticipating',
            'anxious',
            'apprehensive',
            'ashamed',
            'caring',
            'confident',
            'content',
            'devastated',
            'disappointed',
            'disgusted',
            'embarrassed',
            'excited',
            'faithful',
            'furious',
            'grateful',
            'guilty',
            'hopeful',
            'impressed',
            'jealous',
            'joyful',
            'lonely',
            'nostalgic',
            'prepared',
            'proud',
            'sad',
            'sentimental',
            'surprised',
            'terrified',
            'trusting',
            'agreeing', # neutral in emotion
            'acknowledging', # neutral in emotion
            'encouraging', # neutral in emotion
            'consoling', # neutral in emotion
            'sympathizing', # neutral in emotion
            'suggesting', # neutral in emotion
            'questioning', # neutral in emotion
            'wishing', # neutral in emotion
            'neutral'] # neutral in emotion

In [ ]:
import math

# Multi-Head Attention
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dropout_rate, name = 'multi_head_attention'):
        super().__init__(name = name)

        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model, name = 'query')
        self.wk = tf.keras.layers.Dense(d_model, name = 'key')
        self.wv = tf.keras.layers.Dense(d_model, name = 'value')

        self.dropout = tf.keras.layers.Dropout(dropout_rate, name = 'mha_dropout')
        self.dense = tf.keras.layers.Dense(d_model, name = 'mha_output')

    def split_heads(self, x, batch_size):
        """
        Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm = [0, 2, 1, 3])

    def scaled_dot_product_attention(self, q, k, v, mask):
        """
        Calculate the attention weights.
        q, k, v must have matching leading dimensions.
        k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
        The mask has different shapes depending on its type(padding or look ahead) 
        but it must be broadcastable for addition.
        Args:
            q: query shape == (..., seq_len_q, depth)
            k: key shape == (..., seq_len_k, depth)
            v: value shape == (..., seq_len_v, depth_v)
            mask: Float tensor with shape broadcastable 
                to (..., seq_len_q, seq_len_k). Defaults to None.
        
        Returns:
            output, attention_weights
        """

        matmul_qk = tf.matmul(q, k, transpose_b = True)  # (..., seq_len_q, seq_len_k)

        # scale matmul_qk
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # add the mask to the scaled tensor.
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)  

        # softmax is normalized on the last axis (seq_len_k) so that the scores
        # add up to 1.
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)  # (..., seq_len_q, seq_len_k)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        # (As claimed in the RoBERTa implementation.)
        attention_weights = self.dropout(attention_weights)

        output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

        return output, attention_weights

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


def gelu(x):
    """
    Implementation of the gelu activation function.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + tf.math.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * tf.math.pow(x, 3))))
        Also see https://arxiv.org/abs/1606.08415
    """
    return x * 0.5 * (1.0 + tf.math.erf(x / math.sqrt(2.0)))

act_funcs = {'gelu': gelu, 'relu': tf.nn.relu}

# Pointwise Feed Forward Network
def point_wise_feed_forward_network(d_model, dff, hidden_act):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation = act_funcs[hidden_act],
            name = 'ff_hidden'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model, name = 'ff_output')  # (batch_size, seq_len, d_model)
    ], name = 'ff_network')


# Encoder Layer
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, hidden_act, dropout_rate, layer_norm_eps, layer_num):
        super().__init__(name = 'encoder_layer_{:02d}'.format(layer_num))

        self.mha = MultiHeadAttention(d_model, num_heads, dropout_rate)
        self.ffn = point_wise_feed_forward_network(d_model, dff, hidden_act)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = layer_norm_eps,
            name = 'layernorm_1')
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = layer_norm_eps,
            name = 'layernorm_2')

        self.dropout1 = tf.keras.layers.Dropout(dropout_rate, name = 'dropout_1')
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate, name = 'dropout_2')

    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training = training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

In [ ]:
def loss_function(real_emot, pred_emot):
    scce = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits = True, reduction = 'none')
    loss_ = scce(real_emot, pred_emot)
    return loss_

class EmoBERT(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, hidden_act, dropout_rate,
                 layer_norm_eps, max_position_embed, vocab_size, num_emotions):
        super().__init__(name = 'emo_bert')

        self.padding_idx = 1

        # Embedding layers
        self.word_embeddings = tf.keras.layers.Embedding(vocab_size, d_model, name = 'word_embed')
        self.pos_embeddings = tf.keras.layers.Embedding(max_position_embed, d_model, name = 'pos_embed')
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon = layer_norm_eps,
            name = 'layernorm_embed')
        self.dropout = tf.keras.layers.Dropout(dropout_rate, name = 'dropout_embed')

        # Encoder layers
        self.num_layers = num_layers
        self.enc_layers = [
            EncoderLayer(d_model, num_heads, dff, hidden_act, dropout_rate, layer_norm_eps, i)
            for i in range(num_layers)
        ]

        # Output layers
        self.attention_v = tf.keras.layers.Dense(1, use_bias = False, name = 'attention_v')
        self.attention_layer = tf.keras.layers.Dense(d_model, activation = 'tanh', name = 'attention_layer')
        self.hidden_layer = tf.keras.layers.Dense(d_model, activation = 'tanh', name = 'hidden_layer')
        self.output_layer = tf.keras.layers.Dense(num_emotions, name = 'output_layer')

    def call(self, x, training, mask):
        # x.shape == (batch_size, seq_len)

        seq_len = tf.shape(x)[1]

        # Add word embedding and position embedding.
        pos = tf.range(self.padding_idx + 1, seq_len + self.padding_idx + 1)
        pos = tf.broadcast_to(pos, tf.shape(x))
        x = self.word_embeddings(x)  # (batch_size, seq_len, d_model)
        x += self.pos_embeddings(pos)

        x = self.layernorm(x)
        x = self.dropout(x, training = training)

        # x.shape == (batch_size, seq_len, d_model)
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        # Compute the attention scores
        projected = self.attention_layer(x)  # (batch_size, seq_len, d_model)
        logits = tf.squeeze(self.attention_v(projected), 2)  # (batch_size, seq_len)
        logits += (tf.squeeze(mask) * -1e9)  # Mask out the padding positions
        scores = tf.expand_dims(tf.nn.softmax(logits), 1)  # (batch_size, 1, seq_len)

        # x.shape == (batch_size, d_model)
        x = tf.squeeze(tf.matmul(scores, x), 1)

        x = self.hidden_layer(x)
        x = self.output_layer(x)

        return x  # (batch_size, num_emotions)

In [ ]:
# Masking
def create_padding_mask(seq):
    # To be consistent with RoBERTa, the padding index is set to 1.
    seq = tf.cast(tf.math.equal(seq, 1), tf.float32)

    # Add extra dimensions so that we can add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_masks(inp):
    enc_padding_mask = create_padding_mask(inp)
    return enc_padding_mask

def build_model(model, max_length, vocab_size):
    inp = np.ones((1, max_length), dtype = np.int32)
    inp[0,:max_length//2] = np.random.randint(2, vocab_size, size = max_length//2)
    inp = tf.constant(inp)
    enc_padding_mask = create_masks(inp)
    _ = model(inp, True, enc_padding_mask)


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, peak_lr, total_steps, warmup_steps):
        super(CustomSchedule, self).__init__()

        self.peak_lr = peak_lr
        self.total_steps = total_steps
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step / self.warmup_steps
        arg2 = (self.total_steps - step) / (self.total_steps - self.warmup_steps)
        return self.peak_lr * tf.math.minimum(arg1, arg2)

In [11]:
import csv
import numpy as np
from pytorch_transformers import RobertaTokenizer

num_layers = 12
d_model = 768
num_heads = 12
dff = d_model * 4
hidden_act = 'gelu'  # Use 'gelu' or 'relu'
dropout_rate = 0.1
layer_norm_eps = 1e-5
max_position_embed = 514
num_emotions = 41  # Number of categories

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
vocab_size = tokenizer.vocab_size

max_length = 100  # Maximum number of tokens
buffer_size = 100000
batch_size = 1
num_epochs = 10
peak_lr = 2e-5
total_steps = 7000
warmup_steps = 700
adam_beta_1 = 0.9
adam_beta_2 = 0.98
adam_epsilon = 1e-6

checkpoint_path = '/content/drive/My Drive/Colab Notebooks/checkpoints' # Have to specify path to the checkpoint folder in your Google drive

SOS_ID = tokenizer.encode('<s>')[0]
EOS_ID = tokenizer.encode('</s>')[0]

emobert = EmoBERT(num_layers, d_model, num_heads, dff, hidden_act, dropout_rate,
            layer_norm_eps, max_position_embed, vocab_size, num_emotions)

build_model(emobert, max_length, vocab_size)

learning_rate = CustomSchedule(peak_lr, total_steps, warmup_steps)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1 = adam_beta_1, beta_2 = adam_beta_2,
            epsilon = adam_epsilon)

# Define the checkpoint manager.
ckpt = tf.train.Checkpoint(model = emobert, optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = None)

# Restore the checkpoint at epoch 5 - checkpoint with lowest loss on validation set
#print(ckpt_manager.checkpoints[4])
ckpt.restore(ckpt_manager.checkpoints[4])
print('Checkpoint at epoch 5 restored!!')

100%|██████████| 456318/456318 [00:00<00:00, 1710255.27B/s]


Checkpoint at epoch 5 restored!!


# New Section

In [ ]:
import tqdm
import numpy as np

def predict_emotion(uttrs):

    bs = 1

    uttr_ids = np.ones((len(uttrs), max_length), dtype = np.int32)
    i = 0
    u = uttrs[0]
    u_ids = [SOS_ID] + tokenizer.encode(u)[:(max_length-2)] + [EOS_ID]
    uttr_ids[i, :len(u_ids)] = u_ids

    uttr_emots = np.zeros((len(uttrs), num_emotions))
    num_batches = len(uttrs) // bs
    i = 0
    s = i * bs
    t = s + bs
    inp = tf.constant(uttr_ids[s:t])
    enc_padding_mask = create_masks(inp)
    pred = emobert(inp, False, enc_padding_mask)
    pred = tf.nn.softmax(pred).numpy()

    return pred[0]

In [13]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 4.8MB/s eta 0:00:01


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot

from sklearn import model_selection, linear_model, preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from scipy.stats import norm,skewnorm,pearsonr

from bs4 import BeautifulSoup, SoupStrainer
import requests
import time

import pickle
import os
import time
import nltk
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# Download The clean Data set
Solution =  pd.read_pickle("/content/drive/My Drive/Colab Notebooks/BERT/Solution9.pkl")

In [ ]:
# Array type of the labels
labels = np.array(['afraid',#Fear
            'angry',#anger
            'annoyed',#anger
            'anticipating',#Anticipation
            'anxious',#Fear
            'apprehensive',#Fear
            'ashamed',#Disapproval
            'caring',#love
            'confident',#Trust
            'content',#Joy
            'devastated',#Sadness
            'disappointed',#Sadness
            'disgusted',#Disgust
            'embarrassed',#Disapproval
            'excited',#Joy
            'faithful',#Submission
            'furious',#anger
            'grateful',#Trust
            'guilty',#Remorse
            'hopeful',#Anticipation
            'impressed',#Suprise
            'jealous',#Agressiveness
            'joyful',#Joy
            'lonely',#Sadness
            'nostalgic',#Sadness
            'prepared',#Anticipation
            'proud',#Optimism
            'sad',#Sadness
            'sentimental',#Sadness
            'surprised',#Suprise
            'terrified',#Fear
            'trusting',#Trust
            'agreeing', # neutral
            'acknowledging', # neutral 
            'encouraging', # neutral 
            'consoling', # neutral 
            'sympathizing', # neutral 
            'suggesting', # neutral 
            'questioning', # neutral 
            'wishing', # neutral
            'neutral']) # neutral 


In [ ]:
# MAPPING in array type 
plutchik = np.array(['Fear',
            'anger',
            'anger',
            'Anticipation',
            'Fear',
            'Fear',
            'Disapproval',
            'love',
            'Trust',
            'Joy',
            'Sadness',
            'Sadness',
            'Disgust',
            'Disapproval',
            'Joy',
            'Submission',
            'anger',
            'Trust',
            'Remorse',
            'Anticipation',
            'Suprise',
            'Agressiveness',
            'Joy',
            'Sadness',
            'Sadness',
            'Anticipation',
            'Optimism',
            'Sadness',
            'Sadness',
            'Suprise',
            'Fear',
            'trust',
            'neutral',
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral', 
            'neutral',
            'neutral']) # neutral 

In [16]:
# Overview of the data set
Solution

,Num_dialog,character,Text,size
0,489,Kove,"Trent , report . Have you taken out the tar...",11
1,489,Trent,"Negative . He 's got your burn book , and h...",22
2,489,Kove,You don 't have to tell me what would happen...,49
3,489,Alphonse,I can 't tell them apart . They look exactl...,12
4,489,Kove,"Okay , then , shoot them both .",8
...,...,...,...,...
279785,8819128,NOEL,"Roz ... take my hand . Fine , Noel . I 'll...",21
279786,8819128,GIL,"Roz ! Oh , my God ! Put on a towel , you p...",57
279787,8819128,KENNY,"Oh , I 'm about to faint . I 'm going to gr...",48
279788,8819129,ROZ,It 's weird . My skin tastes kinda salty .,10


In [ ]:
# To build a Subset of 100 dialogs (OPTIONAL)
liste_num = Solution["Num_dialog"].unique()[0:100]
Subset = Solution[Solution["Num_dialog"].isin(liste_num)].copy()

In [ ]:
# Vader Analyse
analyzer = SentimentIntensityAnalyzer()
Solution["Vader"] = Solution.Text.apply(lambda x: analyzer.polarity_scores(x)['compound'])

In [ ]:
# Apply Emo_Bert on every Row.

Solution["Emo_bert"] = Solution.Text.apply(lambda x: predict_emotion([x]))

In [ ]:
# Sort the Emobert distribution
Solution["Emo_bert_distribution"]=Solution["Emo_bert"].apply(lambda x: np.sort(x)[-41:][::-1])

# Get the id Of Emobert
Solution["Emo_bert_ids"] = Solution["Emo_bert"].apply(lambda x: x.argsort()[-41:][::-1])

Solution.drop(['Emo_bert'], axis=1, inplace = True)

# Get top 3 Emo_bert
Solution["Emo_Bert"] = Solution["Emo_bert_ids"].apply(lambda x : labels[x[0:3]] )

# Get top 3 plutchik
Solution["plutchik_basic"] = Solution["Emo_bert_ids"].apply(lambda x : plutchik[x[0:3]] 

In [ ]:

print("There is : ",Subset.shape[0]," Dialogs int hte cleanest dataset")
Subset.head(10)

There is :  599  Dialogs int hte cleanest dataset


,Num_dialog,character,Text,size,Vader,Emo_bert_distribution,Emo_bert_ids,Emo_Bert,plutchik_basic
0,489,Kove,"Trent , report . Have you taken out the tar...",11,0.0000,"[0.9971392, 0.00047829366, 0.00021201778, 0.00...","[38, 37, 5, 25, 32, 0, 19, 28, 39, 8, 14, 24, ...","[questioning, suggesting, apprehensive]","[neutral, neutral, Fear]"
1,489,Trent,"Negative . He 's got your burn book , and h...",22,-0.7783,"[0.17218097, 0.16828224, 0.06885416, 0.0656215...","[32, 1, 16, 33, 0, 5, 2, 10, 21, 3, 27, 38, 18...","[agreeing, angry, furious]","[neutral, anger, anger]"
2,489,Kove,You don 't have to tell me what would happen...,49,-0.3612,"[0.8479019, 0.055057667, 0.013935825, 0.012555...","[38, 37, 5, 19, 35, 32, 0, 34, 33, 21, 14, 1, ...","[questioning, suggesting, apprehensive]","[neutral, neutral, Fear]"
3,489,Alphonse,I can 't tell them apart . They look exactl...,12,0.0000,"[0.8424796, 0.02681017, 0.02561466, 0.01869111...","[32, 2, 8, 0, 29, 21, 33, 5, 31, 4, 12, 3, 19,...","[agreeing, annoyed, confident]","[neutral, anger, Trust]"
4,489,Kove,"Okay , then , shoot them both .",8,-0.1280,"[0.24304041, 0.23562402, 0.15192373, 0.0590209...","[37, 35, 32, 40, 34, 33, 38, 1, 31, 30, 0, 10,...","[suggesting, consoling, agreeing]","[neutral, neutral, neutral]"
5,489,Trent,"Wait , what ?",4,0.0000,"[0.9697731, 0.018852782, 0.0019101008, 0.00118...","[38, 37, 5, 14, 32, 35, 19, 3, 0, 4, 1, 33, 29...","[questioning, suggesting, apprehensive]","[neutral, neutral, Fear]"
6,489,Alphonse,What ? !,3,0.0000,"[0.9773402, 0.005786574, 0.0050276015, 0.00205...","[38, 37, 5, 0, 14, 4, 32, 1, 25, 27, 28, 23, 3...","[questioning, suggesting, apprehensive]","[neutral, neutral, Fear]"
7,489,Kove,"Look , Sam and I can 't sit around like Ross...",38,-0.8360,"[0.15415718, 0.15093501, 0.128627, 0.11329802,...","[1, 37, 38, 32, 35, 40, 8, 25, 16, 15, 19, 31,...","[angry, suggesting, questioning]","[anger, neutral, neutral]"
8,489,Sam,Why don 't we use tranqs ?,7,0.0000,"[0.9483739, 0.032491576, 0.0022247022, 0.00182...","[38, 37, 5, 23, 24, 29, 28, 19, 1, 27, 14, 4, ...","[questioning, suggesting, apprehensive]","[neutral, neutral, Fear]"
10,492,Sam,I always said he had a lot of ... hang-ups .,11,0.0000,"[0.19633222, 0.11917248, 0.077162735, 0.060186...","[40, 6, 2, 11, 21, 18, 20, 13, 5, 29, 12, 32, ...","[neutral, ashamed, annoyed]","[neutral, Disapproval, anger]"
